In [1]:
from minicheetah_motor_module_copy import MotorModuleController 
import time
import numpy as np


mmc = MotorModuleController()

sw = 0

In [2]:
mmc.connectSerial(115200, 0)

Serial is connected


1

In [3]:
mmc.read_serial()

In [8]:
mmc.ser.flush()
b = mmc.ser.read(6)
len(b)

0

In [9]:

def getRx():
    try:
        mmc.ser.flush()
        b_rx = mmc.ser.read(6)
        if len(b_rx) == 6:
            for i in range (len(b_rx)):
                mmc.rx_data[i] = b_rx[i]  
        
    except:
        mmc.rx_data = None
    return mmc.rx_data
getRx()

[0, 0, 0, 0, 0, 0]

In [10]:
if sw==0:
    mmc.enable_motor(1)
    sw = 1
else:
    mmc.disable_motor(1)
    sw = 0
time.sleep(0.05)
mmc.read_serial() == None



motor {} enable cmd sent.
 1
motor not connected


True

In [161]:
id = 1
p_des = np.pi*0
v_des = 20
kp = 400
kd = 40
i_ff = 20


mmc.V_MAX = 65

cmd_pack = mmc.pack_cmd(id, p_des, v_des, kp, kd, i_ff)
mmc.send_cmd()
time.sleep(0.2)
rep = mmc.read_serial()
print('cmd',mmc.unpack_cmd())
print('reply', mmc.unpack_reply(rep))
mmc.rx_data

recieved:  [1, 133, 173, 126, 167, 238]
cmd [1, 0.0001907377737087046, 20.015873015873012, 400.0, 5.0, 19.995115995115995]
reply [1, 0.554474708171206, -0.6825396825396837, -0.34188034188034067]


[1, 133, 173, 126, 167, 238]

0 -> 360 deg: 0 -> 0.82 P_des

In [729]:
mmc.ser.read()

b''

In [341]:
mmc.set_zero(1)
mmc.tx_data
0xFE, 0xFC

motor {} set zero cmd sent.
 1


(254, 252)

In [48]:
mmc.unpack_cmd()
mmc.tx_data

[1, 13, 80, 137, 217, 153, 153, 144, 0]

In [80]:
dp = 0.01
while p_des < mmc.P_MAX-0.5:
    st = time.time()
    p_des +=  dp
    mmc.pack_cmd(id, p_des, v_des, kp, kd, i_ff)
    mmc.send_cmd()
    time.sleep(0.001)
    print(mmc.unpack_reply(mmc.read_serial()))
    et = time.time()
    print('time: ', et-st)
while p_des > -(mmc.P_MAX-0.5):
    st = time.time()
    p_des -= dp
    mmc.pack_cmd(id, p_des, v_des, kp, kd, i_ff)
    mmc.send_cmd()
    time.sleep(0.001)
    print(mmc.unpack_reply(mmc.read_serial()))
    et = time.time()
    print('time: ', et-st)

recieved:  [1, 9, 171, 127, 248, 12]
[1, -11.555848020141909, -0.015873015873012264, 0.24420024420024333]
time:  0.0022306442260742188
recieved:  [1, 9, 171, 126, 232, 20]
[1, -11.555848020141909, -0.5555555555555571, 0.40048840048839907]
time:  0.002107381820678711
recieved:  [1, 9, 172, 129, 8, 29]
[1, -11.555466544594491, 0.5238095238095184, 0.5763125763125743]
time:  0.002052783966064453
recieved:  [1, 9, 173, 129, 8, 45]
[1, -11.555085069047074, 0.5238095238095184, 0.8888888888888857]
time:  0.002060413360595703
recieved:  [1, 9, 174, 128, 136, 66]
[1, -11.554703593499656, 0.26984126984126533, 1.2991452991453016]
time:  0.002027750015258789
recieved:  [1, 9, 175, 129, 8, 83]
[1, -11.554322117952239, 0.5238095238095184, 1.6312576312576326]
time:  0.0020570755004882812
recieved:  [1, 9, 179, 129, 72, 95]
[1, -11.55279621576257, 0.6507936507936449, 1.8656898656898662]
time:  0.0023050308227539062
recieved:  [1, 9, 187, 132, 104, 95]
[1, -11.54974441138323, 2.238095238095241, 1.865689

In [776]:
mmc.enable_motor(1)
rep = mmc.ser.read(6)

motor {} enable cmd sent.
 1


In [778]:
print(rep)
len(rep)

b'\x01o\xf4\x7f\xf7\xfe'


6

In [17]:

mmc.read_serial()

IndexError: index out of range

In [16]:
v = 128<<4 | 231>>4
v = uint2float(v, mmc.V_MIN, mmc.V_MAX, 12)

t = (231&0xF)<<8 | 250
t = uint2float(t, -mmc.I_MAX, mmc.I_MAX, 12)

v, t

(0.46031746031745513, -0.10744810744810707)

In [3]:
f'{0.0:d}'

ValueError: Unknown format code 'd' for object of type 'float'

In [29]:
def unpack_tx_data(tx):
    id = tx[0]
    p = tx[1]<<8 | tx[2]
    v = tx[3]<<4 | tx[4]>>4
    kp = ((tx[4]&0xF)<<8) | tx[5]
    kd = tx[6]<<4 | tx[7]>>4
    i = (tx[7]&0xF)<<8 | tx[8]
    return id, p, v, kp, kd, i

In [30]:
id_, p_, v_, kp_, kd_, i_ = unpack_tx_data(cmd_pack)
print(id_, p_, v_, kp_, kd_, i_)

1 58982 3024 819 819 2662


In [31]:
# decode P
p__ = uint2float(p_, mmc.P_MIN, mmc.P_MAX, 16)
p__

10.00019073777371

In [32]:
v__ = uint2float(v_, mmc.V_MIN, mmc.V_MAX, 12)
v__

31.0

In [33]:
kp__ = uint2float(kp_, mmc.KP_MIN, mmc.KP_MAX, 12)
kp__

100.0

In [34]:
kd__ = uint2float(kd_, mmc.KD_MIN, mmc.KD_MAX, 12)
kd__

1.0

In [35]:
i__ = uint2float(i_, -mmc.I_MAX, mmc.I_MAX, 12)
i__

12.004884004884005

In [107]:
mmc.tx_data[8] = 0xFF
mmc.tx_data[7] = 0x01
mmc.tx_data[6] = 0x00
mmc.tx_data[5] = 0x24
mmc.tx_data[4] = 0x00
mmc.tx_data[3] = 0x80
mmc.tx_data[2] = 0x00
mmc.tx_data[1] = 2
print(mmc.tx_data)
mmc.unpack_cmd()


[1, 2, 0, 128, 0, 36, 0, 1, 255]


(1,
 -12.304684519722286,
 0.015873015873012264,
 4.395604395604396,
 0.0,
 -30.017094017094017)

In [18]:
'{0:X}'.format(258)

'102'